In [ ]:
library(dplyr)
library(readr)
library(stringr)
library(tidygraph)
library(ggraph)
library(igraph)
library(ggiraph)
library(here)

In [ ]:
# Load the graph
childcare  <-
  read_csv(here("./data/raw/structural_network_adjacency_list_20190301.csv"),
           col_types = cols(
             source_base_path = col_character(),
             sink_base_path = col_character(),
             link_type = col_character()
           )) %>%
  distinct() %>%
  filter(str_detect(paste(source_base_path, sink_base_path), fixed("childcare"))) %>%
  filter(str_detect(paste(sink_base_path), fixed("childcare"))) %>%
  as_tbl_graph() %>%
  activate(nodes) %>%
  mutate(base_name = str_remove(name, "[\\?#].*$")) %>%
  mutate(base_name = str_remove(base_name, "^.*\\/"))

In [ ]:
# Use the largest component
childcare_main <-
  childcare %>%
  mutate(group = group_components(type = "weak")) %>%
  group_by(group) %>%
  mutate(group_size = n()) %>%
  ungroup() %>%
  filter(group_size == max(group_size))

In [ ]:
# Calculate communities and centrality
childcare_plot_data <-
  childcare_main %>%
  activate("nodes") %>%
  mutate(group = group_spinglass(spins = 5)) %>%
  mutate(centrality = centrality_degree(mode = "out")) %>%
  group_by(group) %>%
  mutate(is_central = centrality == max(centrality)) %>%
  ungroup()

In [ ]:
p <- 
  childcare_plot_data %>%
  ggraph(layout = "nicely") +          # fr and kk are okay too
  geom_point_interactive(aes(x, y, 
     tooltip = name, 
     colour = as.character(group),
     alpha = as.integer(is_central)),
 size = 10) +
  geom_edge_link(arrow = grid::arrow(length = unit(0.05, "inches"), type = "closed"),
    end_cap = circle(0.4, 'cm'),
    start_cap = circle(0.4, 'cm')) +
  geom_node_text(aes(label = centrality)) +
  scale_alpha(range = c(0.2, 1)) +
  theme_graph() +
  nowt()

In [ ]:
girafe(ggobj = p, width_svg = 10, height_svg = 10,
       options = list(opts_sizing(rescale = FALSE)))